In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#let's write a super simple MD system.

In [9]:
def run_simulation():
    import hoomd
    import hoomd.md
    hoomd.context.initialize("")
    #create snapshot
    snapshot = hoomd.data.make_snapshot(N=5,
                                        box = hoomd.data.boxdim(Lx=10, Ly=10,Lz = 10),
                                        particle_types = ['A'],
                                        bond_types=['dna']
                                       )
    snapshot.particles.position[:] = [
        [0,0,0],
        [1,0,0],
        [2,0,0],
        [3,0,0],
        [4,0,0]
            ]
    snapshot.particles.typeid[:] = 0
    
    snapshot.bonds.resize(5)
    snapshot.bonds.group[:] = [[0,1],[1,2],[2,3],[0,4],[3,4]]
    
    hoomd.init.read_snapshot(snapshot)
    
    nl = hoomd.md.nlist.cell()
    
    
    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl);
    harmonic = hoomd.md.bond.harmonic()
    harmonic.bond_coeff.set('polymer', k=100.0, r0=1);
    
    
    hoomd.md.integrate.mode_standard(dt=0.005);

    all = hoomd.group.all();
    hoomd.md.integrate.langevin(group=all, kT=0.2, seed=42);
    
    hoomd.analyze.log(filename="log-output.log",
                  quantities=['potential_energy', 'temperature'],
                  period=100,
                  overwrite=True);

    hoomd.dump.gsd("trajectory.gsd", period=2e3, group=all, overwrite=True);
    hoomd.run(1e4);


In [10]:
run_simulation()

notice(2): Group "all" created containing 5 particles
notice(2): integrate.langevin/bd is using specified gamma values


**ERROR**: Type pair ('A', 'A') not found in pair coeff
**ERROR**: Not all pair coefficients are set


RuntimeError: Error updating pair coefficients